<a href="https://colab.research.google.com/github/zietho/machine-learning/blob/master/Machine_Learning_00.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib as plt
import numpy as np
import re

# Data Set 1: sentiment140

Abstract: This is the sentiment140 dataset. It contains 1,600,000 tweets extracted using the twitter api . The tweets have been annotated (0 = negative, 4 = positive) and they can be used to detect sentiment. <sup>1</sup>

<sup>1</sup> https://www.kaggle.com/kazanova/sentiment140

In [ ]:
df_sentiment = pd.read_csv('datasets/sentiment/training.1600000.processed.noemoticon.csv', encoding='latin_1', header=None)

In [ ]:
df_sentiment.head(2)

In [ ]:
def preprocess(df):

    '''
    weekday extraction 1min
    '''
    def funcapply(x):
        return x[0:3]

    df['weekday'] = df['date'].apply(lambda x: funcapply(x))

    '''
    parse time to pandas datetime takes 5 minutes
    from datetime import datetime
    d = datetime.strptime('Thu Apr 23 13:38:19 +0000 2009','%a %b %d %H:%M:%S %z %Y').strftime('%Y-%m-%d %H:%M:%S');
    '''
    df['date'] = pd.to_datetime(df['date'])
    
    return df
  
df_sentiment = preprocess(df_sentiment)

In [ ]:
df_sentiment.shape

In [ ]:
df_sentiment.head(10)

In [ ]:
df_sentiment.columns = ['label', 'id', 'timestamp', 'query_type', 'username', 'sentiment']

In [ ]:
df_sentiment['label'].value_counts()

In [ ]:
df_sentiment['query_type'].value_counts()

# Data Set 1: OPPORTUNITY Activity Recognition Dataset

Abstract: "The OPPORTUNITY Dataset for Human Activity Recognition from Wearable, Object, and Ambient Sensors is a dataset devised to benchmark human activity recognition algorithms (classification, automatic data segmentation, sensor fusion, feature extraction, etc). <sup>1</sup>

<sup>1</sup> https://archive.ics.uci.edu/ml/datasets/OPPORTUNITY+Activity+Recognition#

# activity of daily living (ADL)
Read in activity of daily living (ADL) for all users

In [ ]:
# read in column names
opportunity_cols = dict()
opportunity_cols_txt = open("datasets/opportunity/column_names.txt", "r")

for line in opportunity_cols_txt:

    if re.search('Column',line) != None:
        col_id = re.search('Column: (\S*) ',line).group(1)
        opportunity_cols[col_id] = dict()
        opportunity_cols[col_id]['name'] = re.search('Column: \S* (\S*)',line).group(1)

        if re.search('Column: \S* \S* \S*',line) != None:
            opportunity_cols[col_id]['sensor'] = re.search('Column: \S* \S* (\S*)',line).group(1)
            opportunity_cols[col_id]['sensor_axis'] = re.search(' (\S*);',line).group(1)
            opportunity_cols[col_id]['value_type'] = re.search('value = (.*),',line).group(1)
            opportunity_cols[col_id]['unit'] = re.search('unit =(.*)$',line).group(1)
            
opportunity_cols

In [ ]:
col_names = list()
for key, column in opportunity_cols.items():
    col_names.append(key+"_"+column.get('name'))
col_names.insert(0,'0_User')
col_names

In [ ]:
adl_filename_mask = 'S{}-ADL{}.dat'
drill_filename_mask = 'S{}-Drill.dat'
df_opportunity_adl = pd.DataFrame()
df_opportunity_drill = pd.DataFrame()

for user_idx in range(1,5):
    for run in range (1,6):
        path = 'datasets/opportunity/'+adl_filename_mask.format(user_idx, run)
        df_partial_adl = pd.read_csv(path, header=None, sep='\s')
        df_partial_adl.insert(0,'User',user_idx)
        df_opportunity_adl = df_opportunity_adl.append(df_partial_adl) 
    
    path = 'datasets/opportunity/'+adl_filename_mask.format(user_idx, run)
    df_partial_drill = pd.read_csv(path, header=None, sep='\s')
    df_partial_drill.insert(0,'User',user_idx)
    df_opportunity_drill = df_opportunity_drill.append(df_partial_adl)

print(df_opportunity_adl.shape)

#df_adl = pd.read_csv('drive/My Drive/University/Data Science/Machine Learning/datasets/opportunity/S1-ADL1.dat', encoding='latin_1', header=None, sep='\s')

In [ ]:
df_opportunity_adl.columns = col_names
#df_opportunity_drill.shape

In [ ]:
df_adl.head(10)